In [1]:
from torchvision.datasets import ImageFolder
import torch
from torchvision import transforms
from architecture import *
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from collections import OrderedDict

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )
transformer = transforms.Compose([
    transforms.Resize([256, 256]),
    # you can add other transformations in this list
    transforms.ToTensor(),
    normalize,
])
dataset = ImageFolder(root="../../DATASETS/", transform=transformer)
n = len(dataset)
n_test = int(0.1 * n)  # take ~10% for test
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-n_test, n_test])

In [3]:
print(len(dataset))
print(len(test_set))
print(len(train_set))
print(test_set)

1669
166
1503


In [4]:
BATCH_SIZE = 16


train_dataloader  = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4) 
test_dataloader  = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4) 

In [5]:

batch_size = 128
learning_rate = 1e-3
pretrained_model = models.resnet101(pretrained=True)
for param in pretrained_model.parameters():
    param.requires_grad = False

pretrained_model.fc = nn.Sequential(
    nn.Linear(2048, 1000),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1000, 4),
    nn.LogSoftmax(dim=1)
)
pretrained_model.to(device)
net = pretrained_model
def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()

In [6]:
print("NET : "+str(net))
input = torch.randn(3, 5, requires_grad=True)
# every element in target should have 0 <= value < C
target = torch.tensor([1, 0, 4])

m = nn.LogSoftmax(dim=1)
nll_loss = nn.NLLLoss()
output = nll_loss(m(input), target)
output.backward()

print("LogSoftMax shape : "+str(m(input).shape))
print("Target shape : "+str(target.shape))

print('input: ', input)
print('target: ', target)
print('output: ', output)

NET : ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [9]:
EPOCHS = 50
LEARNING_RATE = 0.0001
import gc
#del variables
gc.collect()
torch.cuda.empty_cache()
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)
loss_func = nn.NLLLoss()  
running_loss = 0
print_every = 5
# Training and Testing
for epoch in range(EPOCHS):
    print("Epoch : "+str(epoch))
    for step, (x, y) in enumerate(train_dataloader):
        
        b_x = Variable(x).to(device)   # batch x (image)
        b_y = Variable(y).to(device)   # batch y (target)
        
        output = net(b_x)#.argmax(dim=1)
        """
        print("b_y : "+str(b_y))
        print("shape b_y : "+str(b_y.shape))    
        print("output : "+str(output))
        print("shape output : "+str(output.shape))
        """
        loss = loss_func(output, b_y)   
        optimizer.zero_grad()           
        loss.backward()                 
        optimizer.step()
        running_loss += loss.item()

        # Test -> this is where I have no clue
        if step % 10 == 0:
            test_loss = 0
            accuracy = 0
            net.eval()
            with torch.no_grad():
                for inputs, labels in test_dataloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = net(inputs)
                    batch_loss = loss_func(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch}.. "
                  f"Step {step}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(test_dataloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(test_dataloader):.3f}")
            running_loss = 0
            net.train()

Epoch : 0
Epoch 0.. Step 0.. Train loss: 0.274.. Test loss: 1.336.. Test accuracy: 0.326
Epoch 0.. Step 10.. Train loss: 2.819.. Test loss: 1.299.. Test accuracy: 0.496
Epoch 0.. Step 20.. Train loss: 2.627.. Test loss: 1.266.. Test accuracy: 0.472
Epoch 0.. Step 30.. Train loss: 2.549.. Test loss: 1.215.. Test accuracy: 0.468
Epoch 0.. Step 40.. Train loss: 2.438.. Test loss: 1.190.. Test accuracy: 0.509
Epoch 0.. Step 50.. Train loss: 2.479.. Test loss: 1.147.. Test accuracy: 0.551
Epoch 0.. Step 60.. Train loss: 2.491.. Test loss: 1.110.. Test accuracy: 0.589
Epoch 0.. Step 70.. Train loss: 2.354.. Test loss: 1.113.. Test accuracy: 0.547
Epoch 0.. Step 80.. Train loss: 2.514.. Test loss: 1.124.. Test accuracy: 0.595
Epoch 0.. Step 90.. Train loss: 2.249.. Test loss: 1.065.. Test accuracy: 0.616
Epoch : 1
Epoch 1.. Step 0.. Train loss: 0.975.. Test loss: 1.063.. Test accuracy: 0.644
Epoch 1.. Step 10.. Train loss: 2.291.. Test loss: 1.067.. Test accuracy: 0.585
Epoch 1.. Step 20.. Tr

KeyboardInterrupt: 

In [ ]:
torch.save(net.state_dict(), "state_dict")
torch.save(net, "net_model")